## Fine Tuning a Text to spech model

Details of this process are based on the official [documentation](https://tts.readthedocs.io/en/latest/what_makes_a_good_dataset.html#what-makes-a-good-dataset) of TTS library 

In [1]:
!pip install pip install git+https://github.com/statscol/TTS@dev
!pip install gruut-lang-es ## important to install spanish phonemes 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/statscol/TTS (to revision dev) to /tmp/pip-req-build-yv37ozqq
  Running command git clone --filter=blob:none --quiet https://github.com/statscol/TTS /tmp/pip-req-build-yv37ozqq
  Resolved https://github.com/statscol/TTS to commit c5a28dbea5828b089835413287d19d892d600fde
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 8.2 MB/s eta 0:00:00
  Using cached Cython-0.29.28-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_6

In [2]:
import numpy as np 
import pandas as pd 
import os
import logging


In [3]:
## Mount Drive To save files

from google.colab import drive

drive.mount('/content/drive2/')

Mounted at /content/drive2/


In [4]:
## lets also create a folder to save files

import os

DEFAULT_DRIVE_FOLDER="/content/drive2/MyDrive/tts-ai/"
os.makedirs(DEFAULT_DRIVE_FOLDER, exist_ok=True)


## Clone repo with data

In [3]:
!git clone https://github.com/statscol/tts-ai-public-figure.git
%cd tts-ai-public-figure

/content/tts-ai-public-figure


In [6]:
!unzip -q audios_labeled.zip

In [7]:
import re

text="el co2 que emitimos en los 7 continentes"
def num2letters(text):
  n2l={'0':' cero','1':' uno','2':' dos','3':' tres','4':' cuatro','5':' cinco','6':' seis','7':' siete','8':' ocho','9':' nueve'}
  text=re.sub(r"(\d)", lambda x: n2l[str(x.group(0))], text)
  text=re.sub("\s\s+" , " ", text)
  return text

num2letters(text)

'el co dos que emitimos en los siete continentes'

In [8]:
import pandas as pd
import json

data=[json.loads(line) for line in open('manifest.json', 'r')]

data=[f"{i['audio_filepath'].split(chr(92))[-1].replace('.wav','')}|{num2letters(i['text'])}." for i in data]  # chr(92) is backslash
data[:5]

['ea50ff45-0vid1|vengo de uno de los tres países mas bellos de la tierra.',
 '641cca69-0vid2|porque muchas veces el crimen y la violencia disminuyen si.',
 '89444fc2-1vid1|allí hay una explosión de vida miles de.',
 '4c5d5277-1vid2|deja de haber hambre, si hay agua potable, si los jóvenes tienen posibilidades de sexo acc.',
 'ef41bbcf-2vid1|peces multicolores en los mares, en los cielos, en las tierra.']

In [9]:
%mkdir tts-dataset
%mv audio tts-dataset/wavs 

In [10]:
file = open('tts-dataset/metadata.txt','w')
for audio in data:
    file.write(audio+"\n")
file.close()


## lets check the output file

In [4]:
%%bash
tail -10 tts-dataset/metadata.txt

32c1b3ff-161vid1|ado, cuan dependiente son de lo que acabará, co.
77006a13-162vid1|con la especie humana, si observan que los pueblos se llenan de a.
b1d7a02e-163vid1|hambre y de sed, y emigran por millones hacia el norte, has.
e172e4ac-164vid1|y a donde está el agua, entonces ustedes los encierran, constru.
6da9017d-165vid1|truyen muros, despliegan ametralladores, les disparan.
27a3a43b-167vid1|duplican la mentalidad, de quien creó políticamente.
0947d2f6-175vid4|en su sociedad no lo pueden, digamos se tienen que mover muy prudente.
cc028208-177vid4|eh hay, digamos las cifras son contundentes, es que hay un millón.
ca23c690-178vid4|de muertos latinoamericanos, es que millones de personas nam.
ab9ce438-179vid4|mayoría negras, han pasado por consumir o por portar una pequeña cantida.


## Prepare Trainer

Code taken from TTS [documentation](https://tts.readthedocs.io/en/latest/finetuning.html) for fine-tuning

In [10]:
#!tts --list_models #use this to list models
#in the meantime lets use a pre-trained model for spanish

!tts --model_name tts_models/es/mai/tacotron2-DDC --text "Hola"

 > Downloading model to /root/.local/share/tts/tts_models--es--mai--tacotron2-DDC
100% 575M/575M [00:24<00:00, 23.8MiB/s]
 > Model's license - MPL
 > Check https://www.mozilla.org/en-US/MPL/2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--universal--libri-tts--fullband-melgan
100% 109M/109M [00:04<00:00, 22.1MiB/s]
 > Model's license - MPL
 > Check https://www.mozilla.org/en-US/MPL/2.0/ for more info.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | >

In [11]:
from IPython.display import Audio
wn = Audio('tts_output.wav')
display(wn)

### Install loggers and log in

In [5]:
##install logger and instantiate it

#!pip install wandb -q

project = "tts-petro-ai"
display_name = "VITS-es-2"

In [13]:
import wandb 
wandb.login() ##use open session to log in

wandb.init(project=project, name=display_name)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jfparra. Use `wandb login --relogin` to force relogin


In [14]:
##lets see where the model was downloaded
os.listdir("/root/.local/share/tts/tts_models--es--mai--tacotron2-DDC/")

['config.json', 'model_file.pth', 'scale_stats.npy']

### For custom training, we might need to define our own text_cleaner and formatter


In [ ]:
# ## define custom cleaner for spanish , i had to fork the repo and update formatters and cleaners file which include the following methods

# import re

# _whitespace_re = re.compile(r"\s+")

# def replace_symbols(text, lang="en"):
#     """Replace symbols based on the lenguage tag.
#     Args:
#       text:
#        Input text.
#       lang:
#         Lenguage identifier. ex: "en", "fr", "pt", "ca".
#     Returns:
#       The modified text
#       example:
#         input args:
#             text: "si l'avi cau, diguem-ho"
#             lang: "ca"
#         Output:
#             text: "si lavi cau, diguemho"
#     """
#     text = text.replace(";", ",")
#     text = text.replace("-", " ") if lang != "ca" else text.replace("-", "")
#     text = text.replace(":", ",")
#     if lang == "en":
#         text = text.replace("&", " and ")
#     elif lang == "fr":
#         text = text.replace("&", " et ")
#     elif lang == "pt":
#         text = text.replace("&", " e ")
#     elif lang == "ca":
#         text = text.replace("&", " i ")
#         text = text.replace("'", "")
#     elif lang== "es":
#         text=text.replace("&","y")
#         text = text.replace("'", "")
#     return text

# def lowercase(text):
#     return text.lower()

# def collapse_whitespace(text):
#     return re.sub(_whitespace_re, " ", text).strip()

# def remove_aux_symbols(text):
#     text = re.sub(r"[\<\>\(\)\[\]\"]+", "", text)
#     return text

# def spanish_cleaners(text):
#     """Basic pipeline for Portuguese text. There is no need to expand abbreviation and
#     numbers, phonemizer already does that"""
#     text = lowercase(text)
#     text = replace_symbols(text, lang="es")
#     text = remove_aux_symbols(text)
#     text = collapse_whitespace(text)
#     return text

# def ljspeech_custom(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
#     """Normalizes the LJSpeech meta data file to TTS format
#     https://keithito.com/LJ-Speech-Dataset/"""
#     txt_file = os.path.join(root_path, meta_file)
#     items = []
#     speaker_name = "ljspeech"
#     with open(txt_file, "r", encoding="utf-8") as ttf:
#         for line in ttf:
#             cols = line.split("|")
#             wav_file = os.path.join(root_path, "wavs", cols[0] + ".wav")
#             text = cols[1] ## in the repo it appears as cols[2] which created a bug
#             items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
#     return items





## Selecting Model and ModelConfig

For this notebook we'll use **VITS**, however we can also use GlowTTS or Tacotron, but those are more memory-bound and thus could not train them in colab (free tier)

In [7]:
#writefile train_sp_recipe.py

##tacotron needs more ram for training

import os
from trainer import Trainer, TrainerArgs
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.configs.tacotron2_config import Tacotron2Config
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig,CharactersConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor
#import wandb 

DEFAULT_SAMPLE_RATE=16000

data_path="/content/tts-ai-public-figure/"

characters_conf=CharactersConfig(
        pad="<PAD>",
        bos="<BOS>",
        eos="<EOS>",
        blank="<BLNK>",
        characters="abcdefghijklmnopqrstuvwxyzáéíñóú ",
        punctuations="!¡'(),-.:;¿?",
    )

#wandb.login() ##use open session to log in

project = "tts-petro-ai"
display_name = "VITS-es-1"

#wandb_log=wandb.init(project=project, name=display_name)


dataset_config = BaseDatasetConfig(
    formatter="ljspeech_custom",meta_file_train="metadata.txt", path=os.path.join(data_path,"tts-dataset/"))

# INITIALIZE THE TRAINING CONFIGURATION
# Configure the model. Every config class inherits the BaseTTSConfig.

config = VitsConfig(
    run_name=display_name,
    project_name=project,
    batch_size=16,
    eval_batch_size=8,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    save_checkpoints=True,
    save_n_checkpoints=2,
    save_best_after=1000,
    epochs=100,
    characters=characters_conf,
    text_cleaner="spanish_cleaners",
    use_phonemes=True,
    phoneme_language="es-es",
    phoneme_cache_path=os.path.join(DEFAULT_DRIVE_FOLDER,"phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=DEFAULT_DRIVE_FOLDER,
    datasets=[dataset_config],
#    dashboard_logger = 'wandb'
)

# ##AudioProcessor.init_from_config does not allow to set sample rate directly
config.audio['sample_rate']=DEFAULT_SAMPLE_RATE



In [15]:
## Audio processor
ap = AudioProcessor.init_from_config(config)
#=VitsAudioConfig.init_from_config(config)

# INITIALIZE THE TOKENIZER
tokenizer, config = TTSTokenizer.init_from_config(config)
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)


#model = Tacotron2(config, ap, tokenizer, speaker_manager=None)
model=Vits(config, ap, tokenizer, speaker_manager=None)


## see TrainerArgs here (https://github.com/coqui-ai/Trainer/blob/main/trainer/trainer.py)
trainer = Trainer(
    TrainerArgs(restore_path="/content/drive2/MyDrive/tts-ai/VITS-es-1-April-28-2023_08+10PM-7712052/best_model.pth",
                gpu=0), ##default gpu device 0
    config, DEFAULT_DRIVE_FOLDER, model=model, train_samples=train_samples, eval_samples=eval_samples
)


 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 | > Found 405 files in /content/tts-ai-public-figure/tts-dataset


 > Training Environment:
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 > Start Tensorboard: tensorboard --logdir=/content/drive2/MyDrive/tts-ai/VITS-es-1-April-28-2023_09+16PM-7712052
 > Restoring from best_model.pth ...
 > Restoring Model...
 > Restoring Optimizer...
 > Restoring Scaler...
 > Model restored from step 2210

 > Model has 83043436 parameters


## Lets run the training script

In [ ]:
# !CUDA_VISIBLE_DEVICES="0" python train_sp_recipe.py \
#     --restore_path /root/.local/share/tts/tts_models--es--mai--tacotron2-DDC/model_file.pth

trainer.fit()


 > EPOCH: 0/100
 --> /content/drive2/MyDrive/tts-ai/VITS-es-1-April-28-2023_09+16PM-7712052
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > TRAINING (2023-04-28 21:17:52) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: es-es
		| > phoneme backend: gruut
| > Number of instances : 401
 | > Preprocessing samples
 | > Max text length: 94
 | > Min text length: 20
 | > Avg text length: 61.45885286783042
 | 
 | > Max audio length: 80022.0
 | > Min audio length: 64166.0
 | > Avg audio length: 79982.45885286783
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> STEP: 14/26 -- GLOBAL_STEP: 2225
     | > loss_disc: 2.73510  (2.71666)
     | > loss_disc_real_0: 0.17843  (0.19080)
     | > loss_disc_real_1: 0.30627  (0.19283)
     | > loss_disc_real_2: 0.24192  (0.23810)
     | > loss_disc_real_3: 0.25352  (0.23730)
     | > loss_disc_real_4: 0.21376  (0.20456)
     | > loss_disc_real_5: 0.26213  (0.26998)
     | > loss_0: 2.73510  (2.71666)
     | > grad_norm_0: 6.46276  (9.68908)
     | > loss_gen: 1.83010  (1.86350)
     | > loss_kl: 1.73453  (1.67419)
     | > loss_feat: 1.97614  (1.97933)
     | > loss_mel: 25.28261  (24.46132)
     | > loss_duration: 1.82576  (1.74378)
     | > amp_scaler: 512.00000  (3035.42857)
     | > loss_1: 32.64914  (31.72211)
     | > grad_norm_1: 264.70023  (121.62417)
     | > current_lr_0: 0.00020 
     | > current_lr_1: 0.00020 
     | > step_time: 0.89240  (0.81550)
     | > loader_time: 0.01410  (0.00785)


 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: es-es
		| > phoneme backend: gruut
| > Number of instances : 4
 | > Preprocessing samples
 | > Max text length: 81
 | > Min text length: 46
 | > Avg text length: 66.5
 | 
 | > Max audio length: 80022.0
 | > Min audio length: 80022.0
 | > Avg audio length: 80022.0
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.
['<BLNK>', 'i', '<BLNK>', 't', '<BLNK>', ' ', '<BLNK>', 't', '<BLNK>', 'o', '<BLNK>', 'o', '<BLNK>', 'k', '<BLNK>', ' ', '<BLNK>', 'm', '<BLNK>', 'e', '<BLNK>', ' ', '<BLNK>', 'k', '<BLNK>', 'i', '<BLNK>', 't', '<BLNK>', 'e', '<BLNK>', ' ', '<BLNK>', 'a', '<BLNK>', ' ', '<BLNK>', 'l', '<BLNK>', 'o', '<BLNK>', 'n', '<BLNK>', 'ɡ', '<BLNK>', ' ', '<BLNK>', 't', '<BLNK>', 'i', '<BLNK>', 'm', '<BLNK>', 'e', '<BLNK>', ' ', '<BLNK>', 't', '<BLNK>', 'o', '<BLNK>', ' ', '<BLNK>', 'd',


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.47802 (+0.00000)
     | > avg_loss_disc: 2.65182 (+0.00000)
     | > avg_loss_disc_real_0: 0.52053 (+0.00000)
     | > avg_loss_disc_real_1: 0.10750 (+0.00000)
     | > avg_loss_disc_real_2: 0.24460 (+0.00000)
     | > avg_loss_disc_real_3: 0.15851 (+0.00000)
     | > avg_loss_disc_real_4: 0.40067 (+0.00000)
     | > avg_loss_disc_real_5: 0.13709 (+0.00000)
     | > avg_loss_0: 2.65182 (+0.00000)
     | > avg_loss_gen: 2.55073 (+0.00000)
     | > avg_loss_kl: 2.06975 (+0.00000)
     | > avg_loss_feat: 3.07863 (+0.00000)
     | > avg_loss_mel: 25.17411 (+0.00000)
     | > avg_loss_duration: 1.63365 (+0.00000)
     | > avg_loss_1: 34.50686 (+0.00000)

 > BEST MODEL : /content/drive2/MyDrive/tts-ai/VITS-es-1-April-28-2023_09+16PM-7712052/best_model_2237.pth

 > EPOCH: 1/100
 --> /content/drive2/MyDrive/tts-ai/VITS-es-1-April-28-2023_09+16PM-7712052

 > TRAINING (2023-04-28 21:18:38) 


...



> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: es-es
		| > phoneme backend: gruut
	| > 6 not found characters:
	| > ɡ
	| > θ
	| > ͡
	| > ʃ
	| > ʝ
	| > ɾ
| > Number of instances : 401
 | > Preprocessing samples
 | > Max text length: 94
 | > Min text length: 20
 | > Avg text length: 61.45885286783042
 | 
 | > Max audio length: 80022.0
 | > Min audio length: 64166.0
 | > Avg audio length: 79982.45885286783
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> STEP: 20/26 -- GLOBAL_STEP: 4675
     | > loss_disc: 2.52715  (2.59135)
     | > loss_disc_real_0: 0.15509  (0.19472)
     | > loss_disc_real_1: 0.20652  (0.20677)
     | > loss_disc_real_2: 0.27470  (0.23794)
     | > loss_disc_real_3: 0.24868  (0.23669)
     | > loss_disc_real_4: 0.29137  (0.23146)
     | > loss_disc_real_5: 0.23616  (0.25362)
     | > loss_0: 2.52715  (2.59135)
     | > grad_norm_0: 7.03549  (12.21600)
     | > loss_gen: 2.12279  (2.04813)
     | > loss_kl: 1.03211  (1.33699)
     | > loss_feat: 2.73563  (2.62910)
     | > loss_mel: 22.37065  (22.70294)
     | > loss_duration: 1.67120  (1.66727)
     | > amp_scaler: 512.00000  (512.00000)
     | > loss_1: 29.93239  (30.38443)
     | > grad_norm_1: 292.92688  (191.10423)
     | > current_lr_0: 0.00020 
     | > current_lr_1: 0.00020 
     | > step_time: 0.68880  (0.76427)
     | > loader_time: 0.00600  (0.01027)


 > EVALUATION 





> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: es-es
		| > phoneme backend: gruut
	| > 6 not found characters:
	| > ɡ
	| > θ
	| > ͡
	| > ʃ
	| > ʝ
	| > ɾ
| > Number of instances : 4
 | > Preprocessing samples
 | > Max text length: 81
 | > Min text length: 46
 | > Avg text length: 66.5
 | 
 | > Max audio length: 80022.0
 | > Min audio length: 80022.0
 | > Avg audio length: 80022.0
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.79953 (+0.28678)
     | > avg_loss_disc: 2.35082 (-0.34252)
     | > avg_loss_disc_real_0: 0.07744 (-0.10931)
     | > avg_loss_disc_real_1: 0.15084 (-0.07974)
     | > avg_loss_disc_real_2: 0.22542 (-0.01131)
     | > avg_loss_disc_real_3: 0.24327 (-0.01184)
     | > avg_loss_disc_real_4: 0.18859 (-0.05437)
     | > avg_loss_disc_real_5: 0.26229 (+0.01574)
     | > avg_loss_0: 2.35082 (-0.34252)
     | > avg_loss_gen: 2.07280 (+0.16275)
     | > avg_loss_kl: 1.50264 (+0.27209)
     | > avg_loss_feat: 3.32922 (+1.23456)
     | > avg_loss_mel: 22.67395 (+0.99561)
     | > avg_loss_duration: 1.56327 (-0.01479)
     | > avg_loss_1: 31.14190 (+2.65022)


 > EPOCH: 95/100
 --> /content/drive2/MyDrive/tts-ai/VITS-es-1-April-28-2023_09+16PM-7712052

 > TRAINING (2023-04-28 22:13:56) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: es-es
		| > phoneme backend: gruut
	| > 6 not found characters:
	| > ɡ
	| > θ
	| > ͡
	| > ʃ
	| > ʝ
	| > ɾ
| > Number of instances : 401
 | > Preprocessing samples
 | > Max text length: 94
 | > Min text length: 20
 | > Avg text length: 61.45885286783042
 | 
 | > Max audio length: 80022.0
 | > Min audio length: 64166.0
 | > Avg audio length: 79982.45885286783
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


In [6]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [7]:
## Test Model

!tts --text "Hola soy gustavo petro y esta es mi voz ja ja ja" \
      --model_path "/content/drive2/MyDrive/tts-ai/VITS-es-1-April-28-2023_09+16PM-7712052/best_model.pth" \
      --config_path "/content/drive2/MyDrive/tts-ai/VITS-es-1-April-28-2023_09+16PM-7712052/config.json" \
      --out_path output_test.wav


 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: Hola soy gustavo petro y esta es mi voz ja ja ja
 > Text splitted to sentences.
['Hola soy gustavo petro y esta es mi voz ja ja ja']
['<BLNK>', 'o', '<BLNK>', 'l', '<BLNK>', 'a', '<BLNK>', ' ', '<BLNK>', 's', '<BLNK>', 'o', '

In [8]:
from IPython.display import Audio
wn = Audio('output_test.wav')
display(wn)